Задача: обучить модель классификации для цветков Iris'а


In [ ]:
!pip install pyspark   # устанавливаем PySpark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 49.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=63f4778bc7227db4fdd76028a2c34a5e3c90bcfac4588e7dc07af77c82e793f9
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
# загружаем библиотеки
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark_connection
spark = SparkSession(sc) # open spark session

In [ ]:
# загружаем файл Iris через PySpark
# Используем options() чтобы задать несколько опций:
# header=True - Для чтения использует первую строку в качестве имен столбцов
# sep="," - Устанавливает разделитель для каждого поля и значения
# quote="" - определяет, что кавычки не являются разделителем
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')

In [ ]:
df.show(5)   # смотрим, как выглядит файл
# видим, что в заголовках присутствуют кавычки. Уберем их, заменив полностью названия столбцов

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
+-------------+---------------+----------------+---------------+------------+
only showing top 5 rows



In [ ]:
# Чтобы избавиться от кавычек, создадим новый датафрейм
new_columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']
df_new = df.toDF(*new_columns)


In [ ]:
df_new.show(5)   # проверка

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    variety|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|         .2|  Setosa   |
|         4.9|          3|         1.4|         .2|  Setosa   |
|         4.7|        3.2|         1.3|         .2|  Setosa   |
|         4.6|        3.1|         1.5|         .2|  Setosa   |
|           5|        3.6|         1.4|         .2|  Setosa   |
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [ ]:
# видим, что кавычки присутствуют также в 1-м и последнем атрибутах. Уберем их с помощью функции regexp_replace
df_new = df_new.withColumn('sepal_length', regexp_replace(col('sepal_length'), '"', ' '))\
               .withColumn('variety', regexp_replace(col('variety'), '"', ' ')) 

In [ ]:
df_new.show(5)   # проверка

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    variety|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|         .2|  Setosa   |
|         4.9|          3|         1.4|         .2|  Setosa   |
|         4.7|        3.2|         1.3|         .2|  Setosa   |
|         4.6|        3.1|         1.5|         .2|  Setosa   |
|           5|        3.6|         1.4|         .2|  Setosa   |
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [ ]:
# в PySpark по умолчанию формат string. Проверим:
df_new.printSchema()

root
 |-- sepal_length: string (nullable = true)
 |-- sepal_width: string (nullable = true)
 |-- petal_length: string (nullable = true)
 |-- petal_width: string (nullable = true)
 |-- variety: string (nullable = true)



In [ ]:
# для дальнейшей работы с атрибутами нам необходимо привести их к числовому формату. Сделаем это с помощью функции cast
df_new = df_new.withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
               .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
               .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
               .withColumn('petal_width', col('petal_width').cast(DoubleType())) \

In [ ]:
# Проверим, что получилось:
df_new.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [ ]:
# теперь наш датафрейм готов к работе
df_new.show(5)

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    variety|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|  Setosa   |
|         4.9|        3.0|         1.4|        0.2|  Setosa   |
|         4.7|        3.2|         1.3|        0.2|  Setosa   |
|         4.6|        3.1|         1.5|        0.2|  Setosa   |
|         5.0|        3.6|         1.4|        0.2|  Setosa   |
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [ ]:
# посмотрим, на простейшие статистики
df_new.summary().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|       variety|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|          null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|          null|
|    min|               4.3|                2.0|               1.0|               0.1|     Setosa   |
|    25%|               5.1|                2.8|               1.6|               0.3|          null|
|    50%|               5.8|                3.0|               4.3|               1.3|          null|
|    75%|               6.4|                3.3|               5.1|               

In [ ]:
# Поскольку алгоритмы машинного обучения в PySpark принимают на вход только вектора, то нужно провести векторизацию. 
# Для преобразования признаков в вектора используется класс VectorAssembler. 
# Объект этого класса принимает в качестве аргументов список с названиями признаков, которые нужно векторизовать (inputCols), и название преобразованного признака (outputCol).
# После создания объекта VectorAssembler вызывается метод transform.
# Признаки, по которым будет определяться вид: sepal_length, sepal_width, petal_length, petal_width
# Тк признак variety является строковым, а алгоритмы Machine Learning в PySpark работают с числовым значениями, 
# то преобразуем значения столбца variety в числовой вид с помощью StringIndexer
# Указаные шаги можно объединить в Pipeline преобразование и применять как единую операцию:

In [ ]:
pipeline = Pipeline(stages = 
[
  StringIndexer(inputCol='variety', outputCol='varietyInd'),
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')   # используем общепринятое название features
]
)

In [ ]:
# прогоняем через заданный pipeline наш датасет
pipelineTrained = pipeline.fit(df_new)

In [ ]:
# применяем метод transform для объекта VectorAssembler (преобразуем признаки в вектор) 
pipelineTrained.transform(df_new).show()

+------------+-----------+------------+-----------+-----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|    variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-----------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2|  Setosa   |       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|  Setosa   |       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|  Setosa   |       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|  Setosa   |       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|  Setosa   |       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4|  Setosa   |       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3|  Setosa   |       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2|  Setosa   |       

In [ ]:
# с новым признаком Features создаем новый датафрейм
df_features = pipelineTrained.transform(df_new)

In [ ]:
# Далее нам необходимо разбить данные на train и test. 
# В PySpark сделать это очень просто, нужно просто вызвать метод randomSplit, который разделит исходный датасет в заданной пропорции. Мы разделим в пропорции 80:20.

In [ ]:
# назначаем соответствующие веса и указываем seed (рандомное разделение выборки сохранилось)
train, test = df_features.randomSplit([0.8, 0.2], seed=12345)

In [ ]:
# посмотрим на обучающую выборку
train.show()

+------------+-----------+------------+-----------+---------------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|        variety|varietyInd|         Features|
+------------+-----------+------------+-----------+---------------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|      Setosa   |       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|      Setosa   |       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|      Setosa   |       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|      Setosa   |       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|      Setosa   |       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|      Setosa   |       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.4|         1.4|        0.3|      Setosa   |       0.0|[4.6,3.4,1.4,0.3]|
|         4.6|        3.6|    

In [ ]:
# Теперь создадим модель логистической регресии и обучим ее
# Необходимо указать признаки, на которых модель обучается, и признак, который нужно классифицировать.
# Мы преобразовали признаки цветка в вектор под названием features, указываем его в аргументе, а целевым признаком будет числовое поле varietyInd
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

In [ ]:
# Теперь получаем предсказания. Для этого вызывается метод transform, который принимает тестовую выборку, а также обучающую (как обучилась модель):
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [ ]:
# посмотрим, какие предсказания дает модель на обучающей выборке
train_res.show()

+------------+-----------+------------+-----------+---------------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|        variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+---------------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|      Setosa   |       0.0|[4.3,3.0,1.1,0.1]|[67.7027919587531...|[1.0,1.6424718354...|       0.0|
|         4.4|        2.9|         1.4|        0.2|      Setosa   |       0.0|[4.4,2.9,1.4,0.2]|[57.0087434717148...|[1.0,1.4036227809...|       0.0|
|         4.4|        3.0|         1.3|        0.2|      Setosa   |       0.0|[4.4,3.0,1.3,0.2]|[61.7475610236625...|[1.0,2.4772179810...|       0.0|
|         4.4|        3.2|         1.3|        0.2|      Setosa   |       0.0|[4.4,3.2,1.3,0.2]|[68.

In [ ]:
# посмотрим, какие предсказания дает модель на тестовой выборке
test_res.show()

+------------+-----------+------------+-----------+---------------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|        variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+---------------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|      Setosa   |       0.0|[4.6,3.2,1.4,0.2]|[65.8143591752207...|[1.0,8.9932431201...|       0.0|
|         5.0|        3.0|         1.6|        0.2|      Setosa   |       0.0|[5.0,3.0,1.6,0.2]|[52.7723416307807...|[1.0,9.3248066295...|       0.0|
|         5.0|        3.2|         1.2|        0.2|      Setosa   |       0.0|[5.0,3.2,1.2,0.2]|[64.6690738893863...|[1.0,1.7992914773...|       0.0|
|         5.0|        3.5|         1.3|        0.3|      Setosa   |       0.0|[5.0,3.5,1.3,0.3]|[71.

In [ ]:
# теперь необходимо оценить качество предсказания. Для этого воспользуемся MulticlassClassificationEvaluator (так как признаков больше двух)
# Указываем целевой признак varietyInd, а затем вызываем метод evaluate и передаем в него наши предсказания
ev = MulticlassClassificationEvaluator(labelCol = 'varietyInd')

In [ ]:
print('Точность предсказания на контрольной выборке:', ev.evaluate(test_res)*100, '%')
print('Точность предсказания на тренировочной выборке:', ev.evaluate(train_res)*100, '%')

Точность предсказания на контрольной выборке: 100.0 %
Точность предсказания на тренировочной выборке: 98.44961240310077 %


Выводы.

В данном проекте мы обучили модель Machine Learning для решения задачи классификации цветков Ириса. Для этого мы подготовили данные, применили логистическую регрессию и использовали метрики качеств в PySpark.
Модель показала 100% точность на контрольной выборке. 